# Preparing for analysis

## Importing Library

In [1]:
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import matplotlib.pyplot as plt
import datetime
import yfinance as yf
import statsmodels.stats.api as sms
from statsmodels.compat import lzip
from scipy.optimize import minimize
import pytz
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.seasonal import STL
import seaborn as sns
# from sklearn.model_selection import TimeSeriesSplit
import statsmodels.tsa.api as smt
from copy import deepcopy
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import arma_order_select_ic
import warnings
warnings.filterwarnings("ignore")
import statsmodels.tools.eval_measures
import scipy 
from scipy import stats
import time
from matplotlib.dates import DateFormatter
from matplotlib.dates import YearLocator, DateFormatter
from statsmodels.tsa.api import VAR

## Importing Data

In [2]:
df_base = pd.read_excel("./RAW_DATA.xlsx")

In [3]:
df_base

,Date,AAPL,AMZN,BTC-USD,DX-Y.NYB,GC=F,GOOG,MET,MSFT,^GSPC,^TNX,^VIX
0,2014-09-21,25.395000,16.200001,457.334015,84.699997,1234.400024,29.158445,49.269161,46.520000,2001.569946,2.600,12.650000
1,2014-09-28,25.264999,16.225000,402.152008,84.669998,1216.800049,29.288090,49.215687,47.060001,1994.290039,2.566,13.690000
2,2014-10-05,25.027500,16.091000,375.467010,85.589996,1217.500000,28.739098,48.048126,46.439999,1977.800049,2.491,15.980000
3,2014-10-12,24.905001,16.110001,330.079010,85.709999,1206.699951,28.788462,47.228165,46.090000,1964.819946,2.425,15.460000
4,2014-10-19,24.952499,15.322500,390.414001,85.430000,1229.300049,26.587503,43.645275,43.650002,1874.739990,2.286,24.639999
...,...,...,...,...,...,...,...,...,...,...,...,...
473,2023-10-15,178.990005,128.259995,27583.677734,106.080002,1849.500000,139.500000,61.900002,329.820007,4335.660156,4.797,17.700001
474,2023-10-22,178.720001,132.550003,28519.466797,106.239998,1921.099976,140.490005,63.349998,332.640015,4373.629883,4.712,17.209999
475,2023-10-29,173.000000,126.559998,33086.234375,105.540001,1976.300049,137.899994,58.490002,329.320007,4217.040039,4.838,20.370001
476,2023-11-05,170.289993,132.710007,34502.363281,106.120003,1996.199951,125.750000,59.419998,337.309998,4166.819824,4.875,19.750000


# Data Analysis

## Functions

[Function]Making Portofolio

In [4]:
def calculate_efficient_frontier(temp_data, selected):
    # calculate daily and annual returns of the stocks
    returns_monthly = temp_data.pct_change()
    returns_annual = returns_monthly.mean() * 12

    # get daily and covariance of returns of the stock
    cov_monthly = returns_monthly.cov()
    cov_annual = cov_monthly * 12

    # 米国国債のリスクフリーレート
    rf_rate = 0.04849

    # empty lists to store returns, volatility and weights of imaginary portfolios
    port_returns = []
    port_volatility = []
    sharpe_ratio = []
    stock_weights = []

    # set the number of combinations for imaginary portfolios
    num_assets = len(selected)
    num_portfolios = 5000

    np.random.seed(101)

    for single_portfolio in range(num_portfolios):
        weights = np.random.random(num_assets)
        weights /= np.sum(weights)
        returns = np.dot(weights, returns_annual)
        volatility = np.sqrt(np.dot(weights.T, np.dot(cov_annual, weights)))
        sharpe = (returns - rf_rate) / volatility  # シャープレシオの計算を修正
        sharpe_ratio.append(sharpe)
        port_returns.append(returns)
        port_volatility.append(volatility)
        stock_weights.append(weights)

    portfolio = {'Returns': port_returns,
                'Volatility': port_volatility,
                'Sharpe Ratio': sharpe_ratio}

    for counter, symbol in enumerate(selected):
        portfolio[symbol + ' Weight'] = [Weight[counter] for Weight in stock_weights]

    df = pd.DataFrame(portfolio)
    column_order = ['Returns', 'Volatility', 'Sharpe Ratio'] + [stock + ' Weight' for stock in selected]
    df = df[column_order]

    # 最大シャープレシオを持つポートフォリオを探す
    max_sharpe = df['Sharpe Ratio'].max()
    min_volatility = df['Volatility'].min()
    sharpe_portfolio = df.loc[df['Sharpe Ratio'] == max_sharpe]
    min_variance_port = df.loc[df['Volatility'] == min_volatility]

    print(sharpe_portfolio.T)
    print(min_variance_port.T)

    df.plot.scatter(x='Volatility', y='Returns', c='Sharpe Ratio',
                    cmap='RdYlGn', edgecolors='black', figsize=(10, 8), grid=True)
    plt.scatter(x=sharpe_portfolio['Volatility'], y=sharpe_portfolio['Returns'], c='red', marker='D', s=200)
    plt.scatter(x=min_variance_port['Volatility'], y=min_variance_port['Returns'], c='blue', marker='D', s=200)
    plt.xlabel('Volatility (Std. Deviation)')
    plt.ylabel('Expected Returns')
    plt.title('Efficient Frontier with Tangent Portfolio')
    plt.show()

# 使用例:
# calculate_efficient_frontier(temp_data, selected)

[Funstion]ADF test

In [6]:
def perform_adf_test_on_dataframe(df, alpha=0.05):
    # NaNの値を確認し、存在する場合はドロップ
    df.dropna(inplace=True)
    
    # 無限の値を確認し、存在する場合はNaNで置き換えてからドロップ
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)
    
    def adf_test(series):
        # 再度、シリーズにNaNまたは無限の値がないことを確認
        if series.isnull().sum() > 0 or np.isinf(series).sum() > 0:
            raise ValueError(f"Series contains NaN or infinite values")
            
        result = adfuller(series, autolag='AIC')
        test_statistic = result[0]
        p_value = result[1]
        lags_used = result[2]
        nobs = result[3]
        critical_values = result[4]
        result_dict = {
            "ADF Test Statistic": test_statistic,
            "P-Value": p_value,
            "# Lags Used": lags_used,
            "# Observations": nobs,
            "Result": "Stationary" if p_value <= alpha else "Not Stationary"
        }
        for key, value in critical_values.items():
            result_dict[f'Critical Value ({key})'] = value
        return result_dict
    
    results = {col: adf_test(df[col]) for col in df.columns if col != 'Date'}
    return pd.DataFrame(results).T



# Example usage:
# df_results = perform_adf_test_on_dataframe(df_stock_mo)
# display(df_results)

[Function]Deleting Stationality

In [5]:
def remove_trend_diff(data) -> pd.DataFrame:
    """
    Apply differencing and drop NaN rows to remove trend non-stationarity from time series data.
    
    Parameters:
    - data: pd.Series or pd.DataFrame, the input data with one or more time series columns.

    Returns:
    - pd.Series or pd.DataFrame, the transformed data with trend non-stationarity removed.
    """
    
    if isinstance(data, pd.Series):
        data_copy = data.copy()
        return data_copy.diff().dropna()
    
    elif isinstance(data, pd.DataFrame):
        data_copy = data.copy()
        
        # List all columns except 'Date'
        cols_to_diff = [col for col in data_copy.columns if col != 'Date']
        
        # Apply diff() for each column except 'Date'
        for col in cols_to_diff:
            data_copy[col] = data_copy[col].diff()
        
        # Drop rows with NaN values
        return data_copy.dropna()
    
    else:
        raise ValueError("Input data must be a pandas DataFrame or Series")
      
# remove_trend_diff(dataframe)

[Function]Deliting Stationality with Log transformation

In [7]:
def remove_trend_log(data, selected_columns=None) -> pd.DataFrame:
    """
    Compute the log difference for a given DataFrame or Series.
    
    Parameters:
    - data: pd.DataFrame or pd.Series, the input data with one or more time series columns.
    - selected_columns: list, the columns for which the log difference should be computed.
                    If None, it computes for all columns (except 'Date' if present).

    Returns:
    - pd.DataFrame or pd.Series, the log difference of the original data.
    """
    
    # If data is a Series
    if isinstance(data, pd.Series):
        return np.log(data).diff().dropna()
    
    # If data is a DataFrame
    elif isinstance(data, pd.DataFrame):
        
        # If no specific columns are selected, select all columns except 'Date'
        if selected_columns is None:
            selected_columns = [col for col in data.columns if col != 'Date']
        
        # Apply log difference for each selected column
        for col in selected_columns:
            data[col] = np.log(data[col]).diff()
        
        return data.dropna()
    
    else:
        raise ValueError("Input data must be a pandas DataFrame or Series")

# Example usage
# df_log_diff = remove_trend_log(dataframe, ['Column1', 'Column2'])

[Function]VAR model

In [ ]:
'''---- VARモデル -----'''
def create_var_model(data, lags=None):
    """
    データフレームを入力として受け取り、指定されたラグ数でVARモデルをフィットさせる関数
    
    Parameters:
    - data (pd.DataFrame): 多変量の時系列データ
    - lags (int, optional): モデルに使用するラグ数。指定されていない場合、自動的に選択されます。
    
    Returns:
    - model: フィットされたVARモデル
    """

    model = VAR(data)
    
    if lags:
        result = model.fit(lags)
    else:
        result = model.fit(maxlags=12, ic='aic')  # maxlagsと情報量基準(ic)を指定して自動的にラグ数を選択

    return result

# 例:
# data = pd.DataFrame({
#     'y1': np.random.randn(100),
#     'y2': np.random.randn(100)
# })
# fitted_model = create_var_model(data)
# print(fitted_model.summary())


[Function]Forcasting with VAR model

In [8]:

'''---- VARモデルでの予測 ----'''
def plot_var_forecast_for_target(data, target='7203.T', forecast_steps=10, lags=None):
    """
    VARモデルを用いて特定のターゲット変数の予測をグラフで表示する関数
    
    Parameters:
    - data (pd.DataFrame): VARモデルの入力データ
    - target (str): 予測対象のカラム名
    - forecast_steps (int): 予測を行うステップ数
    
    Returns:
    None (グラフを表示する)
    """
    
    # VARモデルをデータに適用
    model = VAR(data)
    if lags:
        fitted_model = model.fit(lags)
    else:
        fitted_model = model.fit(maxlags=12, ic='aic')  # maxlagsと情報量基準(ic)を指定して自動的にラグ数を選択
    fitted_model = model.fit()

    # 予測の生成
    forecast = fitted_model.forecast(data.values[-fitted_model.k_ar:], steps=forecast_steps)
    forecast_df = pd.DataFrame(forecast, columns=data.columns)

    # 予測結果のグラフ表示
    plt.figure(figsize=(10, 6))
    plt.plot(data[target], label='Actual', color='blue')
    plt.plot(np.arange(len(data), len(data) + forecast_steps), forecast_df[target], label='Forecast', color='red')
    plt.title(f'Forecast for {target}')
    plt.legend()
    plt.grid(True)
    plt.show()

# 使用例:
# data = pd.DataFrame({
#     '7203.T': np.random.randn(100),
#     '7201.T': np.random.randn(100)
# })
# plot_var_forecast_for_target(data, target='7203.T', forecast_steps=30)

## Visualizing Data

In [ ]:
## oo